## Model Definition

### Here i am using 6 different kinds of predictive algorithms for predicting this data

##### 1) Polynomial Regression
##### 2) Support Vector Machine Regressor

### Time series predictive models : 

##### 3) Holt's Linear Model
##### 4) AR Model (AUTO ARIMA)
##### 5) Facebook's Prophet Model
## ................................................................................................................

In [63]:
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import timedelta,datetime
from sklearn.svm import SVR
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from pmdarima.arima import auto_arima
from fbprophet import Prophet
import numpy as np

In [64]:
india_datewise = pd.read_csv(os.path.join(os.getcwd(),'Output','india_datewise.csv'))


In [65]:
#the number of days to predict forward
pred_forward_no = 20
pred_forward_no+=1

## Polynomial Regression

polynomial regression is a form of regression analysis in which the relationship between the independent variable x and the dependent variable y is modelled as an nth degree polynomial in x. Polynomial regression fits a nonlinear relationship between the value of x and the corresponding conditional mean of y, denoted E(y |x).

In [66]:
train_ml=india_datewise.iloc[:int(india_datewise.shape[0]*0.95)]
valid_ml=india_datewise.iloc[int(india_datewise.shape[0]*0.95):]

In [67]:
poly = PolynomialFeatures(degree = 6) 

train_poly=poly.fit_transform(np.array(train_ml["Days Since"]).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(valid_ml["Days Since"]).reshape(-1,1))
y_poly = train_ml["Confirmed"]

In [68]:
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,y_poly)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [69]:
model_scores = []
prediction_poly=linreg.predict(valid_poly)
rmse_poly=np.sqrt(mean_squared_error(valid_ml["Confirmed"],prediction_poly))
model_scores.append(rmse_poly)
print("Root Mean Squared Error for Polynomial Regression: ",rmse_poly) 

Root Mean Squared Error for Polynomial Regression:  2165.779250937987


In [70]:
comp_data=poly.fit_transform(np.array(india_datewise["Days Since"]).reshape(-1,1))
plt.figure(figsize=(11,6))
predictions_poly=linreg.predict(comp_data)
fig=go.Figure()
fig.add_trace(go.Scatter(x=india_datewise['ObservationDate'], y=india_datewise["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=india_datewise['ObservationDate'], y=predictions_poly,
                    mode='lines',name="Polynomial Regression Best Fit",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Polynomial Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [71]:
new_date=[]
new_prediction_poly=[]
for i in range(1,pred_forward_no):
    new_date.append(pd.to_datetime(india_datewise['ObservationDate']).iloc[-1]+timedelta(days=i))
    new_date_poly=poly.fit_transform(np.array(india_datewise["Days Since"].max()+i).reshape(-1,1))
    new_prediction_poly.append(linreg.predict(new_date_poly)[0])

In [72]:
model_predictions=pd.DataFrame(zip(new_date,new_prediction_poly),columns=["Date","Polynomial Regression Prediction"])


In [73]:
comp_data=poly.fit_transform(np.array(india_datewise["Days Since"]).reshape(-1,1))
plt.figure(figsize=(11,6))
predictions_poly=linreg.predict(comp_data)
fig=go.Figure()
fig.add_trace(go.Scatter(x=india_datewise['ObservationDate'], y=india_datewise["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=india_datewise['ObservationDate'], y=predictions_poly,
                    mode='lines',name="Polynomial Regression Best Fit",
                    line=dict(color='black', dash='dot')))

fig.add_trace(go.Scatter(x=model_predictions['Date'], y=model_predictions['Polynomial Regression Prediction'],
                    mode='lines+markers',name="Polynomial Regression Prediction"))


fig.update_layout(title="Confirmed Cases Polynomial Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [74]:
model_predictions

,Date,Polynomial Regression Prediction
0,2020-06-07,261195.778811
1,2020-06-08,272852.617358
2,2020-06-09,284949.792312
3,2020-06-10,297501.824303
4,2020-06-11,310523.653163
5,2020-06-12,324030.647298
6,2020-06-13,338038.613175
7,2020-06-14,352563.804919
8,2020-06-15,367622.934030
9,2020-06-16,383233.179204


## Support Vector Machine Regression

“Support Vector Machine” (SVM) is a supervised machine learning algorithm which can be used for both classification or regression challenges. However,  it is mostly used in classification problems. In the SVM algorithm, we plot each data item as a point in n-dimensional space (where n is number of features you have) with the value of each feature being the value of a particular coordinate. Then, we perform classification by finding the hyper-plane that differentiates the two classes very well

In [75]:
svm=SVR(C=0.01,degree=6,kernel='poly')

In [76]:
svm.fit(np.array(train_ml["Days Since"]).reshape(-1,1),train_ml["Confirmed"])

SVR(C=0.01, cache_size=200, coef0=0.0, degree=6, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [77]:
prediction_svm=svm.predict(np.array(valid_ml["Days Since"]).reshape(-1,1))
rmse_svm=np.sqrt(mean_squared_error(prediction_svm,valid_ml["Confirmed"]))
model_scores.append(rmse_svm)
print("Root Mean Square Error for SVR Model: ",rmse_svm)

Root Mean Square Error for SVR Model:  10756.290042808674


In [78]:
plt.figure(figsize=(11,6))
predictions=svm.predict(np.array(india_datewise["Days Since"]).reshape(-1,1))
fig=go.Figure()
fig.add_trace(go.Scatter(x=india_datewise.ObservationDate, y=india_datewise["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=india_datewise.ObservationDate, y=predictions,
                    mode='lines',name="Support Vector Machine Best fit Kernel",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Support Vectore Machine Regressor Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [79]:
new_date=[]
new_prediction_svm=[]
for i in range(1,pred_forward_no):
    new_date.append(pd.to_datetime(india_datewise['ObservationDate']).iloc[-1]+timedelta(days=i))
    new_prediction_svm.append(svm.predict(np.array(india_datewise["Days Since"].max()+i).reshape(-1,1))[0])

In [80]:
model_predictions["SVM Prediction"]=new_prediction_svm


In [81]:
plt.figure(figsize=(11,6))
predictions=svm.predict(np.array(india_datewise["Days Since"]).reshape(-1,1))
fig=go.Figure()
fig.add_trace(go.Scatter(x=india_datewise['ObservationDate'], y=india_datewise["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=india_datewise['ObservationDate'], y=predictions,
                    mode='lines',name="Support Vector Machine Best fit Kernel",
                    line=dict(color='black', dash='dot')))

fig.add_trace(go.Scatter(x=model_predictions['Date'], y=model_predictions['Polynomial Regression Prediction'],
                    mode='lines+markers',name="Polynomial Regression Prediction"))


fig.update_layout(title="Confirmed Cases Support Vectore Machine Regressor Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [82]:
model_predictions.head()

,Date,Polynomial Regression Prediction,SVM Prediction
0,2020-06-07,261195.778811,274162.044666
1,2020-06-08,272852.617358,287163.412214
2,2020-06-09,284949.792312,300674.565550
3,2020-06-10,297501.824303,314711.371838
4,2020-06-11,310523.653163,329290.065793


## Time Series Forecasting Models

Time series analysis comprises methods for analyzing time series data in order to extract meaningful statistics and other characteristics of the data. Time series forecasting is the use of a model to predict future values based on previously observed values.

### Holt's Linear Model

In [83]:
model_train=india_datewise.iloc[:int(india_datewise.shape[0]*0.95)]
valid=india_datewise.iloc[int(india_datewise.shape[0]*0.95):]
print(valid.head())
y_pred=valid.copy()

    ObservationDate  Confirmed  Recovered  Deaths  WeekofYear  Days Since
122      2020-05-31   190609.0    91852.0  5408.0          22         122
123      2020-06-01   198370.0    95754.0  5608.0          23         123
124      2020-06-02   207191.0   100285.0  5829.0          23         124
125      2020-06-03   216824.0   104071.0  6088.0          23         125
126      2020-06-04   226713.0   108450.0  6363.0          23         126


In [84]:
holt=Holt(np.asarray(model_train["Confirmed"])).fit(smoothing_level=0.2, smoothing_slope=2.0)

In [85]:
y_pred["Holt"]=holt.forecast(len(valid))
rmse_holt_linear=np.sqrt(mean_squared_error(y_pred["Confirmed"],y_pred["Holt"]))
model_scores.append(rmse_holt_linear)
print("Root Mean Square Error Holt's Linear Model: ",rmse_holt_linear)

Root Mean Square Error Holt's Linear Model:  4313.125382487645


In [86]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.ObservationDate, y=model_train["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.ObservationDate, y=valid["Confirmed"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid.ObservationDate, y=y_pred["Holt"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases Holt's Linear Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [87]:
holt_new_prediction=[]
for i in range(1,pred_forward_no):
    holt_new_prediction.append(holt.forecast((len(valid)+i))[-1])

model_predictions["Holt's Linear Model Prediction"]=holt_new_prediction


In [88]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.ObservationDate, y=model_train["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.ObservationDate, y=valid["Confirmed"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))

fig.add_trace(go.Scatter(x=model_predictions.Date, y=model_predictions['Holt\'s Linear Model Prediction'],
                    mode='lines+markers',name="Holts Linear Prediction",))
fig.update_layout(title="Confirmed Cases Holt's Linear Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [89]:
model_predictions.head()

,Date,Polynomial Regression Prediction,SVM Prediction,Holt's Linear Model Prediction
0,2020-06-07,261195.778811,274162.044666,247569.896271
1,2020-06-08,272852.617358,287163.412214,256041.771468
2,2020-06-09,284949.792312,300674.565550,264513.646665
3,2020-06-10,297501.824303,314711.371838,272985.521862
4,2020-06-11,310523.653163,329290.065793,281457.397060


### AR Model (using AUTO ARIMA)

In [90]:
model_train=india_datewise.iloc[:int(india_datewise.shape[0]*0.95)]
valid=india_datewise.iloc[int(india_datewise.shape[0]*0.95):]
y_pred=valid.copy()

In [91]:
model_ar= auto_arima(model_train["Confirmed"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=5,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar.fit(model_train["Confirmed"])

Fit ARIMA(0,2,0)x(0,0,0,0) [intercept=True]; AIC=1773.258, BIC=1778.833, Time=0.092 seconds
Fit ARIMA(1,2,0)x(0,0,0,0) [intercept=True]; AIC=1749.602, BIC=1757.965, Time=0.037 seconds
Fit ARIMA(2,2,0)x(0,0,0,0) [intercept=True]; AIC=1749.715, BIC=1760.865, Time=0.061 seconds
Fit ARIMA(3,2,0)x(0,0,0,0) [intercept=True]; AIC=1748.320, BIC=1762.257, Time=0.065 seconds
Fit ARIMA(4,2,0)x(0,0,0,0) [intercept=True]; AIC=1750.230, BIC=1766.955, Time=0.058 seconds
Fit ARIMA(5,2,0)x(0,0,0,0) [intercept=True]; AIC=1752.013, BIC=1771.525, Time=0.258 seconds
Total fit time: 0.590 seconds


ARIMA(maxiter=50, method='lbfgs', order=(3, 2, 0), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [92]:
prediction_ar=model_ar.predict(len(valid))
y_pred["AR Model Prediction"]=prediction_ar

In [93]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"],y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ",np.sqrt(mean_squared_error(y_pred["Confirmed"],y_pred["AR Model Prediction"])))

Root Mean Square Error for AR Model:  2194.2829764491116


In [94]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["Confirmed"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["AR Model Prediction"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases AR Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [95]:
AR_model_new_prediction=[]
for i in range(1,pred_forward_no):
    AR_model_new_prediction.append(model_ar.predict(len(valid)+i)[-1])
model_predictions["AR Model Prediction"]=AR_model_new_prediction

In [96]:
fig=go.Figure()

fig.add_trace(go.Scatter(x=model_predictions.Date, y=model_predictions['AR Model Prediction'],
                    mode='lines+markers',name="AR Model Prediction",))

fig.add_trace(go.Scatter(x=model_train.ObservationDate, y=model_train["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.ObservationDate, y=valid["Confirmed"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.update_layout(title="Confirmed Cases AR Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [97]:
model_predictions.head()

,Date,Polynomial Regression Prediction,SVM Prediction,Holt's Linear Model Prediction,AR Model Prediction
0,2020-06-07,261195.778811,274162.044666,247569.896271,251119.002379
1,2020-06-08,272852.617358,287163.412214,256041.771468,260113.741671
2,2020-06-09,284949.792312,300674.565550,264513.646665,269183.682238
3,2020-06-10,297501.824303,314711.371838,272985.521862,278323.472575
4,2020-06-11,310523.653163,329290.065793,281457.397060,287534.042255


### Facebook's Prophet Model

In [98]:
prophet_c=Prophet(interval_width=0.95,weekly_seasonality=True,)
prophet_confirmed=pd.DataFrame(zip(list(india_datewise.ObservationDate),list(india_datewise["Confirmed"])),columns=['ds','y'])

In [99]:
prophet_c.fit(prophet_confirmed)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/sanjaymsanthosh/anaconda/anaconda3/lib/python3.7/site-packages/fbprophet/forecaster.py:400: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [100]:
forecast_c=prophet_c.make_future_dataframe(periods=pred_forward_no)
forecast_confirmed=forecast_c.copy()

In [101]:
confirmed_forecast=prophet_c.predict(forecast_c)

In [102]:
rmse_prophet=np.sqrt(mean_squared_error(india_datewise["Confirmed"],confirmed_forecast['yhat'].head(india_datewise.shape[0])))
model_scores.append(rmse_prophet)
print("Root Mean Squared Error for Prophet Model: ",rmse_prophet)

Root Mean Squared Error for Prophet Model:  3457.4904795266625


In [103]:
confirmed_forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-339.359594,-7140.278087,6575.437710,-339.359594,-339.359594,26.062374,26.062374,26.062374,26.062374,26.062374,26.062374,0.0,0.0,0.0,-313.297220
1,2020-01-31,-324.237459,-6492.507418,6122.909098,-324.237459,-324.237459,168.172535,168.172535,168.172535,168.172535,168.172535,168.172535,0.0,0.0,0.0,-156.064924
2,2020-02-01,-309.115323,-6437.457076,6557.059026,-309.115323,-309.115323,463.812034,463.812034,463.812034,463.812034,463.812034,463.812034,0.0,0.0,0.0,154.696711
3,2020-02-02,-293.993188,-7229.029889,6045.534099,-293.993188,-293.993188,-245.968953,-245.968953,-245.968953,-245.968953,-245.968953,-245.968953,0.0,0.0,0.0,-539.962141
4,2020-02-03,-278.871053,-7453.768886,6870.209170,-278.871053,-278.871053,-117.246288,-117.246288,-117.246288,-117.246288,-117.246288,-117.246288,0.0,0.0,0.0,-396.117341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2020-06-23,339683.550096,325951.454796,353700.519064,328071.290004,352826.495005,-174.180905,-174.180905,-174.180905,-174.180905,-174.180905,-174.180905,0.0,0.0,0.0,339509.369191
146,2020-06-24,346156.658117,331609.478523,361577.534493,333270.705270,360485.735807,-120.650797,-120.650797,-120.650797,-120.650797,-120.650797,-120.650797,0.0,0.0,0.0,346036.007320
147,2020-06-25,352629.766139,336241.131698,368880.602823,338246.167066,368179.836629,26.062374,26.062374,26.062374,26.062374,26.062374,26.062374,0.0,0.0,0.0,352655.828513
148,2020-06-26,359102.874160,343956.344009,377702.796212,343698.909420,375827.299415,168.172535,168.172535,168.172535,168.172535,168.172535,168.172535,0.0,0.0,0.0,359271.046695


In [104]:
fig=go.Figure()

fig.add_trace(go.Scatter(x=india_datewise.ObservationDate, y=india_datewise.Confirmed,
                    mode='lines',name="FB Prophet Prediction",
                        line=dict(color='black', dash='dot')))

fig.add_trace(go.Scatter(x=confirmed_forecast.ds, y=confirmed_forecast.yhat,
                    mode='lines+markers',name="Train Data for Confirmed Cases"))

fig.update_layout(title="FaceBook Prophet model prediction ",
                 xaxis_title="Date",yaxis_title="Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [105]:

model_predictions["Prophet's Prediction"]=list(confirmed_forecast["yhat"].tail(pred_forward_no-1))
model_predictions["Prophet's Upper Bound"]=list(confirmed_forecast["yhat_upper"].tail(pred_forward_no-1))
model_predictions.head()

,Date,Polynomial Regression Prediction,SVM Prediction,Holt's Linear Model Prediction,AR Model Prediction,Prophet's Prediction,Prophet's Upper Bound
0,2020-06-07,261195.778811,274162.044666,247569.896271,251119.002379,242469.683490,248956.513154
1,2020-06-08,272852.617358,287163.412214,256041.771468,260113.741671,248885.856895,255322.795688
2,2020-06-09,284949.792312,300674.565550,264513.646665,269183.682238,255412.495024,262030.377732
3,2020-06-10,297501.824303,314711.371838,272985.521862,278323.472575,262032.316216,269293.647896
4,2020-06-11,310523.653163,329290.065793,281457.397060,287534.042255,268647.534398,275978.374857


### Summary of models used

In [106]:
models = model_predictions.columns.to_numpy()[1:]
model_evaluation=pd.DataFrame(list(zip(models,model_scores)),columns=["Model Name","Root Mean Squared Error"])
model_evaluation=model_evaluation.sort_values(["Root Mean Squared Error"])
model_evaluation.style.background_gradient(cmap='Reds')

,Model Name,Root Mean Squared Error
0,Polynomial Regression Prediction,2165.779251
3,AR Model Prediction,2194.282976
4,Prophet's Prediction,3457.490480
2,Holt's Linear Model Prediction,4313.125382
1,SVM Prediction,10756.290043


In [107]:
model_predictions["Average of Predictions Models"]=model_predictions.mean(axis=1)
show_predictions=model_predictions.head()
model_predictions

,Date,Polynomial Regression Prediction,SVM Prediction,Holt's Linear Model Prediction,AR Model Prediction,Prophet's Prediction,Prophet's Upper Bound,Average of Predictions Models
0,2020-06-07,261195.778811,274162.044666,247569.896271,251119.002379,242469.683490,248956.513154,254245.486462
1,2020-06-08,272852.617358,287163.412214,256041.771468,260113.741671,248885.856895,255322.795688,263396.699216
2,2020-06-09,284949.792312,300674.565550,264513.646665,269183.682238,255412.495024,262030.377732,272794.093253
3,2020-06-10,297501.824303,314711.371838,272985.521862,278323.472575,262032.316216,269293.647896,282474.692448
4,2020-06-11,310523.653163,329290.065793,281457.397060,287534.042255,268647.534398,275978.374857,292238.511254
5,2020-06-12,324030.647298,344427.255319,289929.272257,296816.735066,275416.281918,282964.345086,302264.089491
6,2020-06-13,338038.613175,360139.927180,298401.147454,306169.975839,281179.608952,288299.188997,312038.076933
7,2020-06-14,352563.804919,376445.452723,306873.022651,315594.612259,287781.439639,295935.533802,322532.310999
8,2020-06-15,367622.934030,393361.593635,315344.897848,325090.533296,294197.613043,302837.892723,333075.910762
9,2020-06-16,383233.179204,410906.507750,323816.773046,334657.491813,300724.251172,309436.659358,343795.810390


### Expected and average plot

In [108]:
fig=go.Figure()

fig.add_trace(go.Scatter(x=india_datewise.ObservationDate, y=india_datewise.Confirmed,
                    mode='lines',name="Confirmed data",
                        line=dict(color='black', dash='dot')))

for val in models:
    fig.add_trace(go.Scatter(x=model_predictions.Date, y=model_predictions[val],
                    mode='lines',name=val,
                            line=dict(dash='dot')))

fig.add_trace(go.Scatter(x=model_predictions.Date, y=model_predictions['Average of Predictions Models'],
                    mode='lines+markers',name="Predicted data",marker = dict(color='black')
                        ))
    

fig.update_layout(title="Model prediction ",
                 xaxis_title="Date",yaxis_title="Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()